In [28]:
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [29]:
# explore data 

import pandas as pd

df = pd.read_csv('/kaggle/input/int3405-sentiment-analysis-problem/full_train.csv')

# print(df.to_string()) 

In [30]:
fea = ['Comment','Rating']
label = []

df = df[fea]

In [31]:
df = df.dropna()

In [32]:
test = pd.read_csv('/kaggle/input/int3405-sentiment-analysis-problem/test.csv')
test.head()

,Unnamed: 0,RevId,UserId,Comment,image_urls
0,0,781115,1326532,Trà táo 35k\nCookie socola 38k \nNước ở đây bì...,['https://images.foody.vn/res/g8/73091/s800/fo...
1,1,1219481,422306,Hôm rồi trung tâm mình tổ chức noel party ở đâ...,['https://images.foody.vn/res/g1/33/s800/foody...
2,2,1703765,9779143,Thịt gà của quán là nhất đấy. Đi ăn gọi liền 4...,['https://images.foody.vn/res/g66/659655/s800/...
3,3,4870346,12924388,Hai đứa ăn xong đau bụng cả ngày\nChân gà ok n...,['https://images.foody.vn/res/g78/772244/s800/...
4,4,2638711,1134279,Mình vừa thử trưa nay. Điểm cộng đầu tiên là b...,['https://images.foody.vn/res/g74/737874/s800/...


In [33]:
import torch
from tqdm import tqdm
device = torch.device("cuda")

In [34]:
model_ckpt = "distilbert-base-uncased"
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt).to(device)
tokenize_input, label = [], []
with torch.no_grad():
  for i in tqdm(range(len(df["Comment"]))): 
      x = df["Comment"].iloc[i] 
      y = df['Rating'].iloc[i]
      # print(i)
      # print("##########################")
      # print(tokenizer.encode(i))
      x = torch.tensor([tokenizer.encode(x)]).to(device)
      if x.shape[1] > 512 : 
        x = x[:,:512]
      # print(x.shape)
      tokenize_input.append(model(x)['last_hidden_state'][0,0,:].to('cpu'))
      label.append(y)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 9070/9070 [00:49<00:00, 185.08it/s]


In [35]:
# dataset 
from torch.utils.data import Dataset, DataLoader
# import pandas as pd

class Review(Dataset): 
    def __init__(self,tokenize_input, label) :
        super().__init__()
        self.tokenize_input = tokenize_input
        self.label = label

    def __len__(self): 
        return len(self.label)

    def __getitem__(self, index):
        return (self.tokenize_input[index], self.label[index])
def loader(tokenize_input, label, batch_size, shuffle, pin_memory=False):
    ds = Review(tokenize_input, label)
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle, pin_memory=pin_memory)

In [36]:
train_loader = loader(tokenize_input, label, batch_size=32, shuffle=True, pin_memory=False)

In [37]:
# model 
import torch.nn as nn

class Encoder(nn.Module): 
    def __init__(self, hidden_dim1, hidden_dim2):
        super().__init__()
        self.linear_1 = nn.Linear( 768 ,hidden_dim1)
        self.linear_2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.linear_3 = nn.Linear(hidden_dim2, 1)
        self.relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.LayerNorm(768)
        self.batchnorm2 = nn.LayerNorm(hidden_dim1)

    def forward(self, input_ids): 
        # print(atac_int.get_device(), gex_int.get_device())
        # with torch.no_grad(): 
        #     features = self.phobert(input_ids)['last_hidden_state']
        # x = features[:, 0 , :]
        # input_ids = self.batchnorm1(input_ids)
        x = self.relu(self.linear_1(input_ids))
        
        x = self.relu(self.linear_2(x))
        # x = self.batchnorm2(x)
        # x = self.dropout(x)
        x = self.linear_3(x)
        return x
        
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [38]:
en = Encoder(128, 64).to(device)

In [39]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [40]:
criterion = nn.BCEWithLogitsLoss()
import torch.optim as optim
optimizer = optim.SGD(model.parameters(), lr=0.0001)
en.train()
EPOCHS = 200 
for e in tqdm(range(1, EPOCHS+1)):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        y_pred = en(X_batch)
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

  0%|          | 1/200 [00:00<01:45,  1.88it/s]

Epoch 001: | Loss: 0.71642 | Acc: 21.254


  1%|          | 2/200 [00:01<01:43,  1.90it/s]

Epoch 002: | Loss: 0.71646 | Acc: 21.215


  2%|▏         | 3/200 [00:01<01:42,  1.93it/s]

...

Epoch 198: | Loss: 0.71643 | Acc: 21.246


100%|█████████▉| 199/200 [01:21<00:00,  2.30it/s]

Epoch 199: | Loss: 0.71646 | Acc: 21.176


100%|██████████| 200/200 [01:22<00:00,  2.43it/s]

Epoch 200: | Loss: 0.71641 | Acc: 21.271


In [41]:
test['Comment'] = test['Comment'].fillna('ngon')
# fill nan vlaue 

In [42]:
test_input = []
with torch.no_grad():
  for i in tqdm(range(len(test["Comment"]))): 
      x = test["Comment"].iloc[i] 
      # y = df['Rating'].iloc[i]
      # print(i)
      # print("##########################")
      # print(tokenizer.encode(i))
      # if pd.isnan()
      x = torch.tensor([tokenizer.encode(x)]).to(device)
      if x.shape[1] > 512 : 
        x = x[:,:512]
      # print(x.shape)
      test_input.append(model(x)['last_hidden_state'][0,0,:].to('cpu'))
      # label.append(y)

100%|██████████| 5103/5103 [00:27<00:00, 183.61it/s]


In [43]:
# predict 
res = []
en = en.to('cpu')
en.eval()
with torch.no_grad():
  for x in tqdm(test_input): 
    y = torch.round(torch.sigmoid(en(torch.unsqueeze(x, 0))))
    res.append(y[0][0].numpy())

100%|██████████| 5103/5103 [00:00<00:00, 8098.02it/s] 


In [1]:
# res.csv is file to submit

test['Rating'] = res
test[['RevId', 'Rating']].to_csv('res.csv', header = ['RevId', 'Rating'], index = False )

NameError: name 'res' is not defined